In [1]:
import sys
sys.path.append('../')

import pandas as pd
from astroquery.sdss import SDSS
from astropy.io import fits
from astropy.table import Table

from stark import sdss

Could not find the pickled WD models. If you need to use these models, please re-import corv with the proper path.
/Users/vedantchandra/0_research/01_sdss5/006_build_corv/data/comm_cat/
star and exposure catalogs not found! check paths and run make_catalogs() if you want to use sdss functionality. otherwise ignore.


In [2]:
catalog = pd.read_csv('../data/raw/Arseneau_2024.csv')

In [4]:
sp = sdss.SDSSHandler(catalog, 'wd_rv_from', '../data/raw/sdss5', '../data/raw/apo')
results = sp.analyze_table('../data/processed/Arseneau_2024.json', lines = ['a', 'b', 'g', 'd'])

  9%|▉         | 35/399 [13:16<2:18:13, 22.79s/it]

In [5]:
catalog.iloc[0]

wd_source_id             78630783787018624
wd_ra                            29.958701
wd_dec                            15.83167
wd_instrument                         BOSS
wd_filepath                            NaN
wd_rv_from                           sdss4
wd_plate                            5119.0
wd_mjd                             55836.0
wd_fiberid                           240.0
wd_rv                            97.570576
wd_e_rv                          39.864918
paths                                  NaN
snr                               5.342997
wd_catalog_id                            0
rv_id                                  0.0
wd_rechi                          0.864229
ms_source_id             78630547564037504
radial_velocity                   5.590984
radial_velocity_error             0.404192
Name: 0, dtype: object